In [1]:
import sys
assert sys.version_info >= (3, 5)
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"
import numpy as np
import time
K = keras.backend
import pandas as pd
import math
import os
from sklearn.metrics import r2_score
from scipy.stats import uniform,randint,norm
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
from sklearn.preprocessing import MinMaxScaler 


#check os.environ ld_library_path is the same here as when I do it in python via terminal, if I get issues

#sometimes I can't select the GPU. In this case, try: https://forums.fast.ai/t/tip-limiting-tensorflow-to-one-gpu/1995

tf.random.set_seed(42)
np.random.seed(42)
random_state=42

In [2]:
save_path = 'save_path'
os.chdir(save_path)

X_train_omics_unlabelled = pd.read_csv("X_train_omics_unlabelled.csv",index_col=0)
X_train_omics_labelled = pd.read_csv("X_train_omics_labelled.csv",index_col=0)
X_test_omics= pd.read_csv("X_test_omics.csv",index_col=0)
X_valid_omics= pd.read_csv("X_valid_omics.csv",index_col=0)
features = np.load("feature_selection.npy",allow_pickle=True)

train_set_labelled_y= pd.read_csv("train_set_labelled_y.csv",index_col=0)
test_set_labelled_y= pd.read_csv("test_set_labelled_y.csv",index_col=0)
valid_set_labelled_y= pd.read_csv("valid_set_labelled_y.csv",index_col=0)

X_train_omics_unlabelled = X_train_omics_unlabelled[features]
X_train_omics_labelled = X_train_omics_labelled[features]
X_test_omics = X_test_omics[features]
X_valid_omics = X_valid_omics[features]

train_set_labelled_c= pd.read_csv("train_set_labelled_c.csv",index_col=0)
train_set_unlabelled_c= pd.read_csv("train_set_unlabelled_c.csv",index_col=0)
test_set_labelled_c= pd.read_csv("test_set_labelled_c.csv",index_col=0)
valid_set_labelled_c= pd.read_csv("valid_set_labelled_c.csv",index_col=0)


train_set_labelled_c = train_set_labelled_c[["age","male"]]
train_set_unlabelled_c = train_set_unlabelled_c[["age","male"]]
test_set_labelled_c = test_set_labelled_c[["age","male"]]
valid_set_labelled_c = valid_set_labelled_c[["age","male"]]

scaler = MinMaxScaler()
train_set_labelled_y = scaler.fit_transform(train_set_labelled_y)
valid_set_labelled_y = scaler.transform(valid_set_labelled_y)
test_set_labelled_y = scaler.transform(test_set_labelled_y)

valid_set_labelled_y[np.where(valid_set_labelled_y >1)] = 1
test_set_labelled_y[np.where(test_set_labelled_y >1)] = 1


train_set_labelled_c["age"] = scaler.fit_transform(train_set_labelled_c[["age"]])
train_set_unlabelled_c["age"] = scaler.transform(train_set_unlabelled_c[["age"]])
test_set_labelled_c["age"] = scaler.transform(test_set_labelled_c[["age"]])
valid_set_labelled_c["age"] = scaler.transform(valid_set_labelled_c[["age"]])

In [7]:
save_model_path = 'save_model_path'
os.chdir(save_model_path)


In [8]:
input_shape = X_train_omics_labelled.shape[1]

# Custom parts # 

## Helpful functions ##

In [9]:
def validation_log_lik_sampling(y_val,x_val,c_val,variational_decoder,codings_size,samples=200):

    """
    Samples a value of z for the expectation, and calculates something proportional to loglikelihood.
    
    The more samples of z, the better the MC approximation to loglik, but the longer it takes to compute.
    
    This is how we do our evaluation on the validation and also test set. 
    
    We look at the ability to generate x given y i.e. loglik(x|y,c)"""
    
    x_val_len = len(x_val)
    expectation = 0
    for i in range(samples):
        z = np.random.normal(loc=0,scale=1,size=codings_size*x_val_len).reshape(x_val_len,codings_size)
        x_pred = variational_decoder([z,y_val,c_val])
        diff = (x_val-x_pred)**2
        pdf = K.sum(diff,axis=-1)
        pdf = K.exp(-pdf)
        expectation += pdf 
    expectation = expectation / samples
    lik = tf.math.log(expectation)
    lik = K.mean(lik)    
    return lik

def create_batch(x_label, y_label, c_label, batch_s=32):
    '''
    Creates batches of labelled and unlabelled data. The total number of points in both batches is equal to batch_s. 
    Thanks to Omer Nivron for help with this.
    
    '''
    proportion_labelled = x_label.shape[0]/(x_label.shape[0])
    
    shape_label = x_label.shape[0]
    label_per_batch = int(np.ceil(proportion_labelled*batch_s))
    batch_idx_la = np.random.choice(list(range(shape_label)), label_per_batch)
    batch_x_la = (x_label.iloc[batch_idx_la, :])
    batch_y_la = (y_label[batch_idx_la,:])
    batch_c_la = (c_label.iloc[batch_idx_la,:])

    
    del batch_idx_la
            
    return batch_x_la, batch_y_la,batch_c_la


def progress_bar(iteration, total, size=30):
    """Progress bar for training"""
    running = iteration < total
    c = ">" if running else "="
    p = (size - 1) * iteration // total
    fmt = "{{:-{}d}}/{{}} [{{}}]".format(len(str(total)))
    params = [iteration, total, "=" * p + c + "." * (size - p - 1)]
    return fmt.format(*params)

def print_status_bar(iteration, total, loss, metrics=None, size=30):
    """Status bar for training"""
    metrics = " - ".join(["Loss for batch: {:.4f}".format(loss)])
    end = "" if iteration < total else "\n"
    print("\r{} - {}".format(progress_bar(iteration, total), metrics), end=end)
    
def print_status_bar_epoch(iteration, total, training_loss_for_epoch,val_loss, metrics=None, size=30):
    """Status bar for training (end of epoch)"""
    metrics = " - ".join(
        ["trainLoss: {:.4f}  Val_loss: {:.4f} ".format(
            training_loss_for_epoch,val_loss)]
    )
    end = "" if iteration < total else "\n"
    print("\r{} - {}".format(progress_bar(iteration, total), metrics), end=end)
    
    
def list_average(list_of_loss):
    return sum(list_of_loss)/len(list_of_loss)
 

def gaussian_pdf(array,mean,sigma):
    part1 = tf.math.divide(tf.constant(np.array(1.0).reshape(1,-1),dtype="float32"),sigma*(2*math.pi)**0.5)
    part2 = K.exp(-0.5*tf.math.divide((array-mean),sigma)**2)
    return part1*part2


## Model ##

### Custom components ###

In [10]:
class Sampling(keras.layers.Layer):
    """reparameterization trick"""
    def call(self, inputs):
        mean, log_var = inputs
        return K.random_normal(tf.shape(log_var)) * K.exp(log_var/2) + mean  
    
    
class y_dist(keras.layers.Layer):

    """
    Custom layer that is used to learn the parameters of the distribution over y.
    
    Outputs a loss. The loss is used for training. The loss is a GMM loss.
    The mean of this is then taken to provide a per batch loss. 
        
    """
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        
        
    def build(self,batch_input_shape):
        self.q1 = self.add_weight(name="q1",shape=[1,1],initializer="random_normal",trainable=True)
        self.q2 = self.add_weight(name="q2",shape=[1,1],initializer="random_normal",trainable=True)
        self.mu1 = self.add_weight(name="mu1",shape=[1,1],initializer="random_normal",trainable=True)
        self.mu2 = self.add_weight(name="mu2",shape=[1,1],initializer="random_normal",trainable=True)
        self.mu3 = self.add_weight(name="mu3",shape=[1,1],initializer="random_normal",trainable=True)
        self.tau1 = self.add_weight(name="tau1",shape=[1,1],initializer="random_normal",trainable=True)
        self.tau2 = self.add_weight(name="tau2",shape=[1,1],initializer="random_normal",trainable=True)
        self.tau3 = self.add_weight(name="tau3",shape=[1,1],initializer="random_normal",trainable=True)

        super().build(batch_input_shape)
    
    def call(self,X):
        concatenated = tf.concat([self.q1,self.q2,tf.constant(np.array(0.0).reshape(1,-1),dtype="float32")],axis=-1)
        p = K.exp(concatenated)
        p = tf.math.divide(p,K.sum(p))
        sigma_concatenated = tf.concat([self.tau1,self.tau2,self.tau3],axis=-1)
        sigma = K.exp(sigma_concatenated)
        likelihood = p[0][0]*gaussian_pdf(X,mean=self.mu1,sigma=sigma[0][0])+p[0][1]*gaussian_pdf(X,mean=self.mu2,sigma=sigma[0][1])+p[0][2]*gaussian_pdf(X,mean=self.mu3,sigma=sigma[0][2]) 
        loglik = tf.math.log(likelihood)
        loss = -loglik
        loss = tf.reduce_mean(loss)
        return loss
    
    def compute_output_shape(self,batch_input_shape):
        return tf.TensorShape(1)    
    
    
class FullModel(keras.models.Model):
    """
    This is the model for labelled data only. For KL. This is used for training purposes.
    
    It requires an encoder, decoder, classifier and y_distribution model to be already defined (as can be done with 
    the build_model function).
    
    It returns the nloglik i.e. the loss. 
    
    This loss can then be used in gradient descent and be minimised wrt parameters (of the four component models).
    
    At test time, you will call which of the component models you want to use (as opposed to trying to "call" this 
    FullModel which you won't want to do as its purpose is just to calculate the nloglik for training).
    
    """
    def __init__(self,N_parameter,beta,variational_encoder,variational_decoder,classifier,y_distribution,
                 **kwargs):
        super().__init__(**kwargs)
        self.encoder = variational_encoder
        self.decoder = variational_decoder
        self.classifier = classifier  
        self.y_distribution = y_distribution
        self.N = N_parameter
        self.beta = beta
    def call(self,inputs):
        """Inputs is a list, as such:
            inputs[0] is labelled X 
            inputs[1] is labelled y 
            inputs[2] is labelled c
        """
        
        X_labelled = inputs[0]
        y_labelled = inputs[1]
        c_labelled = inputs[2]
        
        ############### LABELLED CASE #################
        
        codings_mean,codings_log_var,codings = self.encoder([X_labelled,y_labelled,c_labelled])
        y_pred_mean,y_pred_log_var,y_pred = self.classifier([X_labelled,c_labelled])
        reconstructions = self.decoder([codings,y_labelled,c_labelled])

        #LOSSES#
        recon_loss = labelled_loss_reconstruction(codings_log_var=codings_log_var,x=X_labelled,x_decoded_mean=reconstructions,
                                                  codings_mean=codings_mean,beta=self.beta)
        cls_loss = labelled_cls_loss(y=y_labelled,y_pred_mean=y_pred_mean,y_pred_log_var=y_pred_log_var,N=self.N)
        y_dist_loss1 = self.y_distribution(y_labelled)
        labelled_loss = recon_loss + cls_loss + y_dist_loss1

        
        ############### ALL LOSSES #######################
        
        loss = labelled_loss
        return loss  

    

def build_model(n_hidden=1, n_neurons=723,input_shape=input_shape,beta=1,n_hidden_classifier=1,
              n_neurons_classifier=300,N=30,codings_size=50):
    
    """
    Builds deep generative model.
    
    Parameters specify the architecture. Architecture is such that encoder and decoder have same number of nodes and hidden
    layers. Done for simplicity. Classifier has its own architecture.
    
    Returns encoder,decoder,y_distribution, classifier and overall model. These can be used downstream.
    
    e.g. variational_encoder,variational_decoder,classifier,y_distribution,model = build_model_mmd(n_hidden=1, n_neurons=723,input_shape=input_shape,beta=1,n_hidden_classifier=1,
              n_neurons_classifier=300,N=30,codings_size=50)
    """
       
    ########## ENCODER ###############
    
    x_in = keras.layers.Input(shape=[input_shape])
    y_in = keras.layers.Input(shape=[1])
    c_in = keras.layers.Input(shape=[2])
    z = keras.layers.concatenate([x_in,y_in,c_in])
    for layer in range(n_hidden):
        z = keras.layers.Dense(n_neurons,activation="elu",kernel_initializer="he_normal")(z)
        z = keras.layers.Dropout(0.3)(z)

    codings_mean = keras.layers.Dense(codings_size)(z)
    codings_log_var = keras.layers.Dense(codings_size)(z)
    codings = Sampling()([codings_mean, codings_log_var])
    variational_encoder = keras.models.Model(
        inputs=[x_in,y_in,c_in], outputs=[codings_mean, codings_log_var, codings])
    
    
    ########## DECODER ###############

    latent = keras.layers.Input(shape=[codings_size])
    l_merged = keras.layers.concatenate([latent,y_in,c_in])
    x = l_merged
    for layer in range(n_hidden):
        x = keras.layers.Dense(n_neurons, activation="elu",kernel_initializer="he_normal")(x)
        x = keras.layers.Dropout(0.3)(x)
    x_out = keras.layers.Dense(input_shape,activation="sigmoid")(x) 
    variational_decoder = keras.models.Model(inputs=[latent,y_in,c_in], outputs=[x_out])
    
    
    ########### CLASSIFIER ############
    
    y_classifier = keras.layers.concatenate([x_in,c_in])
    for layer in range(n_hidden_classifier):
        y_classifier = keras.layers.Dense(n_neurons_classifier, activation="elu",kernel_initializer="he_normal")(y_classifier)
        y_classifier = keras.layers.Dropout(rate=0.3)(y_classifier)
        
    y_pred_mean = keras.layers.Dense(1)(y_classifier) 
    y_pred_log_var = keras.layers.Dense(1)(y_classifier) 
    y_pred = Sampling()([y_pred_mean, y_pred_log_var])

    classifier = keras.models.Model(inputs=[x_in,c_in], outputs=[y_pred_mean,y_pred_log_var,y_pred])
    
    
    ############ Y DISTRIBUTION #############
    
    loss = y_dist()(y_in)
    y_distribution = keras.models.Model(inputs=[y_in],outputs=[loss])
    
    
    ########## FULL MODEL #############
    
    model = FullModel(N_parameter=N,beta=beta,variational_encoder=variational_encoder,
                  variational_decoder=variational_decoder,classifier=classifier,y_distribution=y_distribution
                     )
    
    return variational_encoder,variational_decoder,classifier,y_distribution,model

### Loss functions ###

In [11]:
def custom_mse(x,x_decoded_mean):
    """returns column of squared errors. Length of column is number of samples."""
    diff = (x-x_decoded_mean)**2
    return K.sum(diff,axis=-1) /2 

def labelled_loss_reconstruction(codings_log_var,codings_mean,x, x_decoded_mean,beta=1):
    """Labelled data. This is the labelled loss."""
    recon_loss = custom_mse(x, x_decoded_mean)
    kl_loss = - 0.5 * K.sum(1 + codings_log_var - K.square(codings_mean) - K.exp(codings_log_var), axis=-1)
    #kl loss gives vector of one value for each sample in batch 
    return K.mean(recon_loss + beta*kl_loss)


def labelled_cls_loss(y,y_pred_mean,y_pred_log_var,N=383): 
    """Labelled data only. This is the loss function for the part concerning learning the distibution q(y|x). 
        This loss function depends on the mean AND the sigma, hence both are included. This can be derived from
        taking the log of the normal distribution pdf.
        
        N is the number of labelled data points in the training set. It is used with alpha to weight this loss term, so
        that the model learns q(y|x) using the labelled training data. 
    """
    alpha=0.1*N
    sigma = K.exp(y_pred_log_var/2)
    diff = (y-y_pred_mean)**2
    regression_loss = tf.math.divide(diff,(2*sigma**2))
    loss = regression_loss + 0.5*y_pred_log_var
    return alpha*K.mean(loss)

### Training functions ###

In [12]:
@tf.function
def train_step(inputs):
    """Decorated train_step function which applies a gradient update to the parameters"""
    with tf.GradientTape() as tape:
        loss = model(inputs,training=True)
        loss = tf.add_n([loss] + model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

def fit_model(X_train_la, y_train_la,c_train_la,epochs,X_valid_la, y_valid_la,c_valid_la,
              patience,variational_encoder,variational_decoder,
             classifier,y_distribution,model,Sampling=Sampling,y_dist=y_dist,batch_size=32,learning_rate=0.001,codings_size=50,
             valid_set=True):

    """
    Fits a single model. Gets the validation loss too if valid set exists. 
    And includes a version of early stopping, given by the patience.
    Progress bars are shown too.
    Number of epochs are specified by the parameter epochs.
    
    Need to pass in all the custom components. Maybe could put them in a dictionary for cleanliness.
    
    Valid set is True or False depending if you have one. If you don't, the model at the end of training is saved.
    You must still pass in dummy valid sets even if valid_set=False.
    
    Returns list of training loss, and the minimum validation loss. It also saves the best encoder, decoder and
    regressor so they can be used. 
    
    e.g. usage fit_model(X_train_omics_labelled, train_set_labelled_y, X_train_omics_unlabelled,50,X_valid_omics, valid_set_labelled_y,
              10,variational_encoder=variational_encoder,variational_decoder=variational_decoder,
             classifier=classifier,y_distribution=y_distribution,model=model,
          Sampling=Sampling,y_dist=y_dist,batch_size=32,learning_rate=0.001,codings_size=50,valid_set=True)
    """
    if valid_set is True:
    
        start = time.time()
        history = []
        K.clear_session()

        @tf.function
        def train_step(inputs):
            with tf.GradientTape() as tape:
                loss = model(inputs,training=True)
                loss = tf.add_n([loss] + model.losses)
            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            return loss

        validation_loss = []
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate)
        batch_loss = []
        batches_per_epoch = int(np.floor((X_train_la.shape[0])/batch_size))

        for epoch in range(epochs):

                print("Epoch {}/{}".format(epoch,epochs))

                for i in range(batches_per_epoch):

                    batch_x_la, batch_y_la,batch_c_la= create_batch(
                        X_train_la, y_train_la,c_train_la,batch_size)

                    inputs = [batch_x_la.to_numpy(),batch_y_la,batch_c_la.to_numpy()]
                    loss = train_step(inputs)
                    batch_loss.append(loss)
                    average_batch_loss = list_average(batch_loss)
                    print_status_bar(i*batch_size,X_train_la.shape[0],average_batch_loss)

                training_loss_for_epoch = list_average(batch_loss)
                batch_loss = []
                history.append(training_loss_for_epoch)
                val_loss = -validation_log_lik_sampling(y_valid_la,X_valid_la.to_numpy(),c_valid_la.to_numpy(),
                                                        variational_decoder=variational_decoder,codings_size=codings_size)

                validation_loss.append(val_loss)
                print_status_bar_epoch(X_train_la.shape[0]
                                 ,(X_train_la.shape[0]),training_loss_for_epoch,val_loss )

                #callback for early stopping
                if epoch <= patience - 1:

                    if epoch == 0:

                        variational_encoder.save("variational_encoder.h5")
                        variational_decoder.save("variational_decoder.h5")
                        classifier.save("classifier.h5")
                        y_distribution.save("y_distribution.h5")

                    else:
                        if all(val_loss<i for i in validation_loss[:-1]) is True:
                            variational_encoder.save("variational_encoder.h5")
                            variational_decoder.save("variational_decoder.h5")
                            classifier.save("classifier.h5")
                            y_distribution.save("y_distribution.h5")
                #this statement means at least a model is saved. Because if the best model was before epoch > patience-1,
                #then the statement below won't save any model, which is undesirable as we need to load a model. 

                if epoch > patience - 1:

                    latest_val_loss = validation_loss[-patience:]
                    if all(val_loss<i for i in latest_val_loss[:-2]) is True:
                        variational_encoder.save("variational_encoder.h5")
                        variational_decoder.save("variational_decoder.h5")
                        classifier.save("classifier.h5")
                        y_distribution.save("y_distribution.h5")
                    if all(i>latest_val_loss[0] for i in latest_val_loss[1:]) is True:
                        break     

        #load best model#
        variational_encoder = keras.models.load_model("variational_encoder.h5", custom_objects={
           "Sampling": Sampling
        })
        variational_decoder = keras.models.load_model("variational_decoder.h5")
        classifier = keras.models.load_model("classifier.h5", custom_objects={
           "Sampling": Sampling
        })    
        y_distribution = keras.models.load_model("y_distribution.h5", custom_objects={
           "y_dist": y_dist
        })    

        done = time.time()
        elapsed = done-start
        print("Elapsed/s: ",elapsed)
        print("Final training loss: ",training_loss_for_epoch)
        print("best val loss: ", min(validation_loss))
        
        return history, min(validation_loss)

        
    else:
        
        start = time.time()
        history = []
        K.clear_session()

        @tf.function
        def train_step(inputs):
            with tf.GradientTape() as tape:
                loss = model(inputs,training=True)
                loss = tf.add_n([loss] + model.losses)
            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            return loss
        
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate)
        batch_loss = []
        batches_per_epoch = int(np.floor((X_train_la.shape[0])/batch_size))
        val_loss = 0

        for epoch in range(epochs):

                print("Epoch {}/{}".format(epoch,epochs))

                for i in range(batches_per_epoch):

                    batch_x_la, batch_y_la,batch_c_la= create_batch(
                        X_train_la, y_train_la,c_train_la,batch_size)

                    inputs = [batch_x_la.to_numpy(),batch_y_la,batch_c_la.to_numpy()]
                    loss = train_step(inputs)
                    batch_loss.append(loss)
                    average_batch_loss = list_average(batch_loss)
                    print_status_bar(i*batch_size,X_train_la.shape[0],average_batch_loss)

                training_loss_for_epoch = list_average(batch_loss)
                batch_loss = []
                history.append(training_loss_for_epoch)
                print_status_bar_epoch(X_train_la.shape[0]
                                 ,(X_train_la.shape[0]),training_loss_for_epoch,val_loss )
        

        variational_encoder.save("variational_encoder.h5")
        variational_decoder.save("variational_decoder.h5")
        classifier.save("classifier.h5")
        y_distribution.save("y_distribution.h5")
        
        #load best model#
        variational_encoder = keras.models.load_model("variational_encoder.h5", custom_objects={
           "Sampling": Sampling
        })
        variational_decoder = keras.models.load_model("variational_decoder.h5")
        classifier = keras.models.load_model("classifier.h5", custom_objects={
           "Sampling": Sampling
        })     
        y_distribution = keras.models.load_model("y_distribution.h5", custom_objects={
           "y_dist": y_dist
        })    

        done = time.time()
        elapsed = done-start
        print("Elapsed/s: ",elapsed)
        print("Final training loss: ",training_loss_for_epoch)
        
    
        return history


def fit_model_search(X_train_la, y_train_la,c_train_la, epochs,X_valid_la, y_valid_la,c_valid_la,
              patience,variational_encoder,variational_decoder,
             classifier,y_distribution,model,Sampling=Sampling,y_dist=y_dist,batch_size=32,learning_rate=0.001,
                    codings_size=50):

    """
    Use for hyperparameter search. 
    
    Fits the model. Gets the validation loss too. And includes a version of early stopping, given by the patience.
    Progress bars are shown too.
    Number of epochs are specified by the parameter epochs.
    
    Need to pass in all the custom components. Maybe could put them in a dictionary for cleanliness.
    
    Returns list of training loss, and the minimum validation loss. It also saves the best encoder, decoder and
    regressor so they can be used. 
    
    """
    
    start = time.time()
    history = []   
       
    @tf.function
    def train_step(inputs):
        with tf.GradientTape() as tape:
            loss = model(inputs,training=True)
            loss = tf.add_n([loss] + model.losses)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        return loss
    
    validation_loss = []
    optimizer=keras.optimizers.Adam(learning_rate=learning_rate)
    batch_loss = []
    batches_per_epoch = int(np.floor((X_train_la.shape[0])/batch_size))

    for epoch in range(epochs):

            print("Epoch {}/{}".format(epoch,epochs))

            for i in range(batches_per_epoch):

                batch_x_la, batch_y_la,batch_c_la= create_batch(
                    X_train_la, y_train_la,c_train_la,batch_size)

                inputs = [batch_x_la.to_numpy(),batch_y_la,batch_c_la.to_numpy()]
                loss = train_step(inputs)
                batch_loss.append(loss)
                average_batch_loss = list_average(batch_loss)
                print_status_bar(i*batch_size,X_train_la.shape[0],average_batch_loss)

            training_loss_for_epoch = list_average(batch_loss)
            batch_loss = []                
            history.append(training_loss_for_epoch)            
            val_loss = -validation_log_lik_sampling(y_valid_la,X_valid_la.to_numpy(),c_valid_la.to_numpy(),
                                                    variational_decoder=variational_decoder,codings_size=codings_size)

            validation_loss.append(val_loss)            
            print_status_bar_epoch(X_train_la.shape[0] 
                             ,(X_train_la.shape[0] ),training_loss_for_epoch,val_loss )

            #callback for early stopping
            
            if epoch <= patience - 1:
                
                if epoch == 0:
                
                    variational_encoder.save("variational_encoder_intermediate.h5")
                    variational_decoder.save("variational_decoder_intermediate.h5")
                    classifier.save("classifier_intermediate.h5")
                    y_distribution.save("y_distribution_intermediate.h5")
                    
                else:
                    if all(val_loss<i for i in validation_loss[:-1]) is True:
                        variational_encoder.save("variational_encoder_intermediate.h5")
                        variational_decoder.save("variational_decoder_intermediate.h5")
                        classifier.save("classifier_intermediate.h5")
                        y_distribution.save("y_distribution_intermediate.h5")
            #this statement means at least a model is saved. Because if the best model was before epoch > patience-1,
            #then the statement below won't save any model, which is undesirable as we need to load a model. 
            
            if epoch > patience - 1:
                                
                latest_val_loss = validation_loss[-patience:]
                if all(val_loss<i for i in latest_val_loss[:-1]) is True:
                    variational_encoder.save("variational_encoder_intermediate.h5")
                    variational_decoder.save("variational_decoder_intermediate.h5")
                    classifier.save("classifier_intermediate.h5")
                    y_distribution.save("y_distribution_intermediate.h5")
                if all(i>latest_val_loss[0] for i in latest_val_loss[1:]) is True:
                    break     
    
    #load best model#
    variational_encoder = keras.models.load_model("variational_encoder_intermediate.h5", custom_objects={
       "Sampling": Sampling
    })
    variational_decoder = keras.models.load_model("variational_decoder_intermediate.h5")
    classifier = keras.models.load_model("classifier_intermediate.h5", custom_objects={
           "Sampling": Sampling
        })    
    y_distribution = keras.models.load_model("y_distribution_intermediate.h5", custom_objects={
       "y_dist": y_dist
    })    
                
    done = time.time()
    elapsed = done-start
    print("Elapsed/s: ",elapsed)
    print("Final training loss: ",training_loss_for_epoch)
    print("best val loss: ", min(validation_loss))
    
    return history, min(validation_loss)

def hyperparameter_search(param_distribs,epochs,patience,n_iter,X_train_la=X_train_omics_labelled, 
                          y_train_la=train_set_labelled_y, c_train_la=train_set_labelled_c,
                             c_valid_la=valid_set_labelled_c,
                          X_valid_la=X_valid_omics, y_valid_la=valid_set_labelled_y):
    
    """
    Performs hyperparameter, random search. Assesses performance by determining the score on the validation set. 
    
    Saves best models (encoder, decoder and regressor) and returns these. These can then be used downstream.
    
    Also returns dictionary of the search results.
    
    Param_distribs of the form: 
            param_distribs = {
            "n_hidden": [1],
            "n_hidden_classifier": [1],
            "beta": [1],
            "n_neurons": randint.rvs(50,1000-49,size=20,random_state=random_state).tolist(),
           "n_neurons_classifier": randint.rvs(49,1000-49,size=20,random_state=random_state).tolist(),
            "codings_size": randint.rvs(50,290-50,size=30,random_state=random_state).tolist(),
            "N" :randint.rvs().tolist(),
            "learning_rate" : ....
            #"codings_size": [50]}
            
    There must be a value for every parameter. If you know the value you want to use, set it in the param_distribs
    dictionary.
    
    Patience must be less than the number of epochs.
    
    e.g. result,variational_encoder,variational_decoder,classifier,y_distribution =
            hyperparameter_search_mmd(param_distribs,500,10,n_iter=10)

            
    """

    np.random.seed(42) #needs to be here so that everything that follows is consistent

    min_val_loss = []
    master = {}

    for i in range(n_iter): 
        K.clear_session()
        master[i] = {}
        master[i]["parameters"] = {}
        
        N= np.random.choice(param_distribs["N"])
        learning_rate= np.random.choice(param_distribs["learning_rate"])
        beta= np.random.choice(param_distribs["beta"])
        n_neurons =np.random.choice(param_distribs["n_neurons"]) 
        n_neurons_classifier =np.random.choice(param_distribs["n_neurons_classifier"]) 
        n_hidden  =np.random.choice(param_distribs["n_hidden"]) 
        n_hidden_classifier  =np.random.choice(param_distribs["n_hidden_classifier"]) 
        codings_size =np.random.choice(param_distribs["codings_size"]) 
       
        master[i]["parameters"]["N"] = N
        master[i]["parameters"]["learning_rate"] = learning_rate
        master[i]["parameters"]["beta"] = beta
        master[i]["parameters"]["n_neurons"] = n_neurons
        master[i]["parameters"]["n_neurons_classifier"] = n_neurons_classifier
        master[i]["parameters"]["n_hidden"] = n_hidden
        master[i]["parameters"]["n_hidden_classifier"] = n_hidden_classifier
        master[i]["parameters"]["codings_size"] = codings_size

        
        variational_encoder,variational_decoder,classifier,y_distribution,model = build_model(n_hidden=n_hidden,       
                                       n_neurons=n_neurons,beta=beta,n_hidden_classifier=n_hidden_classifier,
                                        n_neurons_classifier=n_neurons_classifier,N=N,codings_size=codings_size)
        
                
        history,val_loss = fit_model_search(X_train_la=X_train_la, y_train_la=y_train_la, 
                                epochs=epochs,X_valid_la=X_valid_la, 
                                 y_valid_la=y_valid_la,patience=patience,variational_encoder=variational_encoder,
                                variational_decoder=variational_decoder, classifier=classifier,
                                y_distribution=y_distribution,model=model,Sampling=Sampling,y_dist=y_dist,
                                            batch_size=32,learning_rate=learning_rate,codings_size=codings_size,
                                            
                                c_train_la=c_train_la,c_valid_la=c_valid_la
                                           )        

        master[i]["val_loss"] = val_loss
        min_val_loss.append(val_loss)

        #If val loss is lowest, save this model. 
        if val_loss <=  min(min_val_loss):
            os.rename("variational_encoder_intermediate.h5","variational_encoder.h5")
            os.rename("variational_decoder_intermediate.h5","variational_decoder.h5")
            os.rename("classifier_intermediate.h5","classifier.h5")
            os.rename("y_distribution_intermediate.h5","y_distribution.h5")

        print(master)
            
    #load best model#
    variational_encoder = keras.models.load_model("variational_encoder.h5", custom_objects={
       "Sampling": Sampling
    })
    variational_decoder = keras.models.load_model("variational_decoder.h5")
    classifier = keras.models.load_model("classifier.h5", custom_objects={
           "Sampling": Sampling
        }) 
    y_distribution = keras.models.load_model("y_distribution.h5", custom_objects={
       "y_dist": y_dist
    })    

    result = sorted(master.items(), key=lambda item: item[1]["val_loss"])
    return result,variational_encoder,variational_decoder,classifier,y_distribution


# Hyperparameter Search #

In [12]:
param_distribs = {
            "n_hidden": [1,2],
            "n_hidden_classifier": [1,2],
            "beta": [1,10,15],
    #"n_neurons": [300,500],
   # "n_neurons_classifier": [50,100,150],
            "n_neurons": randint.rvs(50,600-49,size=20,random_state=random_state).tolist(),
           "n_neurons_classifier": randint.rvs(20,120-20,size=20,random_state=random_state).tolist(),
            "codings_size": randint.rvs(20,290-20,size=30,random_state=random_state).tolist(),
   # "codings_size": [20,50,70],
            "N" :[0.1,1,10
                 
                 
            ],
            "learning_rate" : [0.001,0.0005],
            #"codings_size": [120,60]
}

In [14]:
result,variational_encoder,variational_decoder,classifier,y_distribution = hyperparameter_search(param_distribs=param_distribs,
                                                                        epochs=80,patience=10,n_iter=5)
result

Epoch 0/80

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

320/383 [========================>.....] - Loss for batch: 79.8977WARNING:tensorflow:Layer concatenate_1 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all

383/383 [==============================] - trainLoss: 2.3550  Val_loss: 13.0279 
Epoch 73/80
383/383 [==============================] - trainLoss: 2.3296  Val_loss: 13.0977 
Epoch 74/80
383/383 [==============================] - trainLoss: 2.3722  Val_loss: 13.1333 
Epoch 75/80
383/383 [==============================] - trainLoss: 2.2489  Val_loss: 13.1284 
Epoch 76/80
383/383 [==============================] - trainLoss: 2.2444  Val_loss: 13.1460 
Epoch 77/80
383/383 [==============================] - trainLoss: 2.0861  Val_loss: 13.2129 
Epoch 78/80
383/383 [==============================] - trainLoss: 2.2374  Val_loss: 13.1894 
Epoch 79/80
383/383 [==============================] - trainLoss: 2.1428  Val_loss: 13.1225 
Elapsed/s:  62.3584668636322
Final training loss:  tf.Tensor(2.14285, shape=(), dtype=float32)
best val loss:  tf.Tensor(13.004663, shape=(), dtype=float32)
{0: {'parameters': {'codings_size': 169, 'beta': 1, 'n_neurons': 137, 'learning_rate': 0.0005, 'n_hidden': 2, '

Elapsed/s:  42.21155095100403
Final training loss:  tf.Tensor(8.011768, shape=(), dtype=float32)
best val loss:  tf.Tensor(12.920776, shape=(), dtype=float32)
{0: {'parameters': {'codings_size': 169, 'beta': 1, 'n_neurons': 137, 'learning_rate': 0.0005, 'n_hidden': 2, 'n_hidden_classifier': 1, 'n_neurons_classifier': 72, 'N': 10.0}, 'val_loss': <tf.Tensor: shape=(), dtype=float32, numpy=13.004663>}, 1: {'parameters': {'codings_size': 177, 'beta': 10, 'n_neurons': 152, 'learning_rate': 0.001, 'n_hidden': 2, 'n_hidden_classifier': 2, 'n_neurons_classifier': 40, 'N': 1.0}, 'val_loss': <tf.Tensor: shape=(), dtype=float32, numpy=12.920776>}}
Epoch 0/80

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' t

383/383 [==============================] - trainLoss: 21.6192  Val_loss: 13.9099 
Epoch 40/80
383/383 [==============================] - trainLoss: 20.6272  Val_loss: 13.9095 
Epoch 41/80
383/383 [==============================] - trainLoss: 21.0191  Val_loss: 13.8191 
Epoch 42/80
383/383 [==============================] - trainLoss: 20.0826  Val_loss: 13.7598 
Epoch 43/80
383/383 [==============================] - trainLoss: 20.0524  Val_loss: 13.7119 
Epoch 44/80
383/383 [==============================] - trainLoss: 19.8289  Val_loss: 13.6649 
Epoch 45/80
383/383 [==============================] - trainLoss: 18.9321  Val_loss: 13.6402 
Epoch 46/80
383/383 [==============================] - trainLoss: 18.7467  Val_loss: 13.6028 
Epoch 47/80
383/383 [==============================] - trainLoss: 18.8004  Val_loss: 13.6132 
Epoch 48/80
383/383 [==============================] - trainLoss: 18.4442  Val_loss: 13.6010 
Epoch 49/80
383/383 [==============================] - trainLoss: 17.839

[(4,
  {'parameters': {'N': 0.1,
    'beta': 1,
    'codings_size': 34,
    'learning_rate': 0.001,
    'n_hidden': 2,
    'n_hidden_classifier': 1,
    'n_neurons': 485,
    'n_neurons_classifier': 40},
   'val_loss': <tf.Tensor: shape=(), dtype=float32, numpy=12.424155>}),
 (2,
  {'parameters': {'N': 0.1,
    'beta': 10,
    'codings_size': 149,
    'learning_rate': 0.001,
    'n_hidden': 1,
    'n_hidden_classifier': 1,
    'n_neurons': 380,
    'n_neurons_classifier': 21},
   'val_loss': <tf.Tensor: shape=(), dtype=float32, numpy=12.603399>}),
 (1,
  {'parameters': {'N': 1.0,
    'beta': 10,
    'codings_size': 177,
    'learning_rate': 0.001,
    'n_hidden': 2,
    'n_hidden_classifier': 2,
    'n_neurons': 152,
    'n_neurons_classifier': 40},
   'val_loss': <tf.Tensor: shape=(), dtype=float32, numpy=12.920776>}),
 (0,
  {'parameters': {'N': 10.0,
    'beta': 1,
    'codings_size': 169,
    'learning_rate': 0.0005,
    'n_hidden': 2,
    'n_hidden_classifier': 1,
    'n_neurons':

Best KL-based model with only labelled samples has val nloglik of 12.424155. 

 {'parameters': {'N': 0.1,
    'beta': 1,
    'codings_size': 34,
    'learning_rate': 0.001,
    'n_hidden': 2,
    'n_hidden_classifier': 1,
    'n_neurons': 485,
    'n_neurons_classifier': 40},
   'val_loss': <tf.Tensor: shape=(), dtype=float32, numpy=12.424155

# Single run # 

In [13]:
variational_encoder,variational_decoder,classifier,y_distribution,model = build_model(n_hidden=2, n_neurons=485,input_shape=input_shape,beta=1,n_hidden_classifier=1,
              n_neurons_classifier=40,N=0.1,codings_size=34)

In [14]:
fit_model(X_train_omics_labelled, train_set_labelled_y,train_set_labelled_c,
          100,X_valid_omics, valid_set_labelled_y,valid_set_labelled_c,
              10,variational_encoder=variational_encoder,variational_decoder=variational_decoder,
             classifier=classifier,y_distribution=y_distribution,model=model,
          Sampling=Sampling,y_dist=y_dist,batch_size=32,learning_rate=0.001,valid_set=True,codings_size=34)

Epoch 0/100

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

320/383 [========================>.....] - Loss for batch: 41.8404WARNING:tensorflow:Layer concatenate_1 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change al

([<tf.Tensor: shape=(), dtype=float32, numpy=41.840443>,
  <tf.Tensor: shape=(), dtype=float32, numpy=19.390163>,
  <tf.Tensor: shape=(), dtype=float32, numpy=14.61428>,
  <tf.Tensor: shape=(), dtype=float32, numpy=11.665411>,
  <tf.Tensor: shape=(), dtype=float32, numpy=10.405419>,
  <tf.Tensor: shape=(), dtype=float32, numpy=9.5561695>,
  <tf.Tensor: shape=(), dtype=float32, numpy=8.738458>,
  <tf.Tensor: shape=(), dtype=float32, numpy=8.201592>,
  <tf.Tensor: shape=(), dtype=float32, numpy=7.527142>,
  <tf.Tensor: shape=(), dtype=float32, numpy=7.1215496>,
  <tf.Tensor: shape=(), dtype=float32, numpy=6.82744>,
  <tf.Tensor: shape=(), dtype=float32, numpy=6.4796176>,
  <tf.Tensor: shape=(), dtype=float32, numpy=6.2714977>,
  <tf.Tensor: shape=(), dtype=float32, numpy=5.9635024>,
  <tf.Tensor: shape=(), dtype=float32, numpy=5.697788>,
  <tf.Tensor: shape=(), dtype=float32, numpy=5.677863>,
  <tf.Tensor: shape=(), dtype=float32, numpy=5.432315>,
  <tf.Tensor: shape=(), dtype=float32, n

In [15]:
test_nlog_lik = -validation_log_lik_sampling(test_set_labelled_y,X_test_omics.to_numpy(),test_set_labelled_c.to_numpy(),
                                    variational_decoder=variational_decoder,codings_size=34,samples=2000)
print("test_nlog_lik = " + str(test_nlog_lik))

test_nlog_lik = tf.Tensor(13.229876, shape=(), dtype=float32)
